# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)

In [ ]:
#from library.controller.sql_controller import SqlController
#from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point
#from library.atlas.brain_structure_manager import BrainStructureManager
#from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
#    read_image, write_image


In [ ]:
def get_coms(animal):
    """
    Fetches the COMs from disk. The data is stored in micrometers.
    """
    
    coms = {}
    dirpath = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}/origin'
    if not os.path.exists(dirpath):
        return coms
    files = sorted(os.listdir(dirpath))
    for file in files:
        structure = Path(file).stem
        filepath = os.path.join(dirpath, file)
        com = np.loadtxt(filepath)
        com *= 10
        coms[structure] = com
    return coms


def compute_affine_transformation(src_pts, dst_pts):
    """
    src_pts: Nx3 array of points in brain A
    dst_pts: Nx3 array of points in brain B
    
    Returns:
        T  : 4x4 affine matrix mapping src → dst
        err: RMS error after transform
    """

    src_pts = np.asarray(src_pts)
    dst_pts = np.asarray(dst_pts)
    assert src_pts.shape == dst_pts.shape
    assert src_pts.shape[0] >= 4  # minimum points for 3D affine

    N = src_pts.shape[0]

    # Build homogeneous design matrix: [x y z 1]
    X = np.hstack([src_pts, np.ones((N, 1))])        # Nx4
    Y = dst_pts                                       # Nx3

    # Solve X * A = Y  → A is 4×3 matrix
    A, *_ = np.linalg.lstsq(X, Y, rcond=None)

    # Convert to 4×4 transform
    T = np.eye(4)
    T[:3, :4] = A.T

    # Apply transform to src
    src_h = np.hstack([src_pts, np.ones((N, 1))])    # Nx4
    dst_pred = (T @ src_h.T).T[:, :3]

    # Compute RMS error
    rmse = np.sqrt(np.mean(np.sum((dst_pts - dst_pred)**2, axis=1)))

    return T, rmse, dst_pred


In [ ]:
moving_name = 'AtlasV8'
fixed_name = 'Allen'
moving_all = get_coms(moving_name)
fixed_all = get_coms(fixed_name)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('10N_L','10N_R')
good_keys = set(common_keys) - set(bad_keys)
moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(good_keys))
transformation_matrix, rmse, predictions = compute_affine_transformation(moving_src, fixed_src)
data_path = "/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration"
transformation_matrix_path = os.path.join(data_path, moving_name, f'{moving_name}_{fixed_name}_10.0um')
np.save(transformation_matrix_path, transformation_matrix)
print('rmse', rmse)

df_list = []
predicted_dict = {}
for structure in good_keys:    
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    predicted0 = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(predicted0, fixed0)]   
    row = [structure, np.round(moving0,2), np.round(fixed0,2), np.round(predicted0,4), np.round(difference,8)]
    df_list.append(row)
    predicted_dict[structure] = predicted0

In [ ]:
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS: 237.06805950085737 observations: 37
# MD585 to MD594 152.06606097021333 observations: 51
# MD585 to Allen 263.31435 observations: 37
# MD589 to Allen 260.02 observations: 37
# MD594 to Allen 250.79 observations: 37

In [ ]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#comtfm = np.array([824.6051918494063, 80.83004570523167, 363.4390121956811])
transformed_structure = affine_transform_point(com, transformation_matrix)

print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
diff = transformed_structure - fixed_all[structure]
#comdiff = comtfm - fixed_all[structure]

print(f'{moving_name}->{fixed_name} error {structure} {diff}')
#print(f'{moving_name}->{fixed_name} tfm error {structure} {comdiff}')

In [ ]:
columns = ['structure', moving_name, fixed_name, 'predicted', 'difference']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(2)
df.sort_values(by=['structure'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)
#20	3N_R	[1079.0, 531.0, 485.0]	[910.0, 380.0, 17.0]	[873.0, 358.0, 298.0]	[-37.0, -21.0, 281.0]	284.3851
#7	4N_L	[1135.0, 529.0, 423.0]	[959.0, 378.0, 544.0]	[923.0, 362.0, 356.0]	[-36.0, -16.0, -188.0]	192.0327

In [ ]:
registered_origin_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/registered_origin'
for structure, origin in moving_all.items():
    new_origin = affine_transform_point(origin, transformation_matrix) / 10
    new_origin_path = os.path.join(registered_origin_path, f'{structure}.txt')
    print(structure, np.round(origin/10,2), np.round(new_origin,2))
    np.savetxt(new_origin_path, new_origin)